# Assignment 2 - Data Preprocessing & Feature Engineering Solutions

This notebook contains complete solutions for all parts of Assignment 2.

**Dataset**: Microsoft Adventure Works Cycles Customer Data

**Objective**: Predict which customers are most likely to purchase a bike

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

---
# Part I: Feature Selection, Cleaning, and Preprocessing

Construct an input dataset from the data source by selecting relevant features.

## Load the Datasets

In [ ]:
# Load customer and sales data
df_customers = pd.read_csv("AWCustomers.csv")
df_sales = pd.read_csv("AWSales.csv")

print(f"Customers Dataset Shape: {df_customers.shape}")
print(f"Sales Dataset Shape: {df_sales.shape}")

print("\nCustomers Columns:")
print(df_customers.columns.tolist())

print("\nSales Columns:")
print(df_sales.columns.tolist())

In [ ]:
# Preview the data
print("Customers Data Preview:")
df_customers.head()

In [ ]:
# Merge datasets
df_sales_clean = df_sales.drop(['CustomerID'], axis=1)
df = pd.concat([df_customers, df_sales_clean], axis=1)

print(f"Merged Dataset Shape: {df.shape}")
df.head()

## Part I(a): Examine and Select Relevant Attributes

Analyze each attribute and select only those that would affect predicting future bike buyers.

In [ ]:
# Examine all columns
print("="*60)
print("ATTRIBUTE ANALYSIS")
print("="*60)

print("\nAll columns in merged dataset:")
for i, col in enumerate(df.columns):
    print(f"  {i+1}. {col} - dtype: {df[col].dtype}")

In [ ]:
# Identify unnecessary attributes that won't help predict bike buyers
unnecessary_cols = [
    'Title',           # Mostly null, not predictive
    'Suffix',          # Mostly null, not predictive
    'MiddleName',      # Not predictive for buying behavior
    'AddressLine2',    # Mostly null
    'PhoneNumber',     # Personal info, not predictive
    'FirstName',       # Personal identifier, not predictive
    'LastName',        # Personal identifier, not predictive
    'AddressLine1',    # Too specific, not generalizable
    'City',            # Too many unique values
    'PostalCode',      # Too specific
    'LastUpdated'      # System metadata, not predictive
]

# Remove unnecessary attributes
df_selected = df.drop(columns=[col for col in unnecessary_cols if col in df.columns])

print("Removed attributes:")
for col in unnecessary_cols:
    print(f"  - {col}")

print(f"\nRemaining attributes: {df_selected.shape[1]}")

## Part I(b): Create New DataFrame with Selected Attributes

In [ ]:
# Create new DataFrame with selected features
print("="*60)
print("SELECTED FEATURES FOR BIKE BUYER PREDICTION")
print("="*60)

print(f"\nSelected columns ({len(df_selected.columns)}):")
for col in df_selected.columns:
    null_count = df_selected[col].isnull().sum()
    print(f"  - {col}: {df_selected[col].dtype}, nulls: {null_count}")

print(f"\nDataFrame shape: {df_selected.shape}")
df_selected.head()

## Part I(c): Determine Data Value Type for Each Attribute

Classify each attribute as:
- **Discrete** or **Continuous**
- **Nominal**, **Ordinal**, **Interval**, or **Ratio**

In [ ]:
# Classify data types
print("="*70)
print("DATA TYPE CLASSIFICATION")
print("="*70)

# Discrete vs Continuous
discrete_vars = []
continuous_vars = []

for col in df_selected.columns:
    if df_selected[col].dtype in ['int64', 'object', 'bool']:
        discrete_vars.append(col)
    else:
        continuous_vars.append(col)

print("\n1. DISCRETE vs CONTINUOUS:")
print(f"   Discrete Variables: {discrete_vars}")
print(f"   Continuous Variables: {continuous_vars}")

In [ ]:
# Nominal, Ordinal, Interval, Ratio classification
nominal_vars = [
    'CustomerID', 'Gender', 'MaritalStatus', 'StateProvinceName', 
    'CountryRegionName', 'Education', 'Occupation', 'HomeOwnerFlag', 'BikeBuyer'
]

ordinal_vars = [
    'Education'  # Has inherent ordering (High School < Bachelors < Graduate)
]

interval_vars = [
    'BirthDate'  # Dates are interval scale
]

ratio_vars = [
    'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 
    'YearlyIncome', 'AvgMonthSpend'
]

print("\n2. NOMINAL / ORDINAL / INTERVAL / RATIO:")
print(f"   Nominal (categorical, no order): {[v for v in nominal_vars if v in df_selected.columns]}")
print(f"   Ordinal (categorical, ordered): {[v for v in ordinal_vars if v in df_selected.columns]}")
print(f"   Interval (numeric, no true zero): {[v for v in interval_vars if v in df_selected.columns]}")
print(f"   Ratio (numeric, true zero exists): {[v for v in ratio_vars if v in df_selected.columns]}")

In [ ]:
# Create summary table
type_summary = []
for col in df_selected.columns:
    disc_cont = 'Discrete' if col in discrete_vars else 'Continuous'
    if col in nominal_vars:
        scale = 'Nominal'
    elif col in ordinal_vars:
        scale = 'Ordinal'
    elif col in interval_vars:
        scale = 'Interval'
    else:
        scale = 'Ratio'
    type_summary.append({'Attribute': col, 'Discrete/Continuous': disc_cont, 'Scale': scale})

type_df = pd.DataFrame(type_summary)
print("\nCOMPLETE DATA TYPE SUMMARY:")
print(type_df.to_string(index=False))

---
# Part II: Data Preprocessing and Transformation

Transform the preprocessed data based on data types.

## Part II(a): Handling Null Values

In [ ]:
print("="*60)
print("HANDLING NULL VALUES")
print("="*60)

# Check null values before
print("\nNull values BEFORE imputation:")
null_counts = df_selected.isnull().sum()
print(null_counts[null_counts > 0])

# Define numerical and categorical features
numerical_features = ['YearlyIncome', 'AvgMonthSpend', 'NumberCarsOwned', 
                      'NumberChildrenAtHome', 'TotalChildren']
categorical_features = ['Gender', 'MaritalStatus', 'HomeOwnerFlag', 'BikeBuyer']

# Impute numerical features with mean
numerical_imputer = SimpleImputer(strategy='mean')
for col in numerical_features:
    if col in df_selected.columns:
        df_selected[col] = numerical_imputer.fit_transform(df_selected[[col]])

# Impute categorical features with mode
categorical_imputer = SimpleImputer(strategy='most_frequent')
for col in categorical_features:
    if col in df_selected.columns:
        df_selected[col] = categorical_imputer.fit_transform(df_selected[[col]]).ravel()

print("\nNull values AFTER imputation:")
print(f"Total nulls remaining: {df_selected.isnull().sum().sum()}")

## Part II(b): Normalization (Min-Max Scaling)

In [ ]:
print("="*60)
print("NORMALIZATION (Min-Max Scaling)")
print("="*60)

# Create a copy for normalized data
df_normalized = df_selected.copy()

# Apply Min-Max normalization to numerical features
min_max_scaler = MinMaxScaler()

print("\nBEFORE Normalization:")
print(df_normalized[numerical_features].describe().loc[['min', 'max']].T)

df_normalized[numerical_features] = min_max_scaler.fit_transform(df_normalized[numerical_features])

print("\nAFTER Normalization (values scaled to 0-1):")
print(df_normalized[numerical_features].describe().loc[['min', 'max']].T)

## Part II(c): Discretization (Binning)

In [ ]:
print("="*60)
print("DISCRETIZATION (Binning)")
print("="*60)

# Apply binning to YearlyIncome (continuous attribute with many values)
binning_transformer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

# Get original YearlyIncome before normalization
original_income = df_selected['YearlyIncome'].copy()

df_normalized['YearlyIncome_Binned'] = binning_transformer.fit_transform(
    df_selected[['YearlyIncome']]
).astype(int)

print("\nYearlyIncome Binning (5 bins using quantiles):")
print(df_normalized['YearlyIncome_Binned'].value_counts().sort_index())

# Show bin edges
bin_edges = binning_transformer.bin_edges_[0]
print(f"\nBin edges: {bin_edges.round(0)}")
print("\nBin labels: 0=Low, 1=Below-Average, 2=Average, 3=Above-Average, 4=High")

## Part II(d): Standardization (Z-Score Normalization)

In [ ]:
print("="*60)
print("STANDARDIZATION (Z-Score)")
print("="*60)

# Apply StandardScaler (mean=0, std=1)
standard_scaler = StandardScaler()

df_standardized = df_selected.copy()
df_standardized[numerical_features] = standard_scaler.fit_transform(
    df_selected[numerical_features]
)

print("\nAFTER Standardization (mean=0, std=1):")
stats = df_standardized[numerical_features].describe().loc[['mean', 'std']].T
print(stats.round(4))

## Part II(e): Binarization (One-Hot Encoding)

In [ ]:
print("="*60)
print("BINARIZATION (One-Hot Encoding)")
print("="*60)

# Columns to one-hot encode
cols_to_encode = ['Gender', 'MaritalStatus']

print("\nBEFORE One-Hot Encoding:")
for col in cols_to_encode:
    if col in df_standardized.columns:
        print(f"  {col}: {df_standardized[col].unique()}")

# Apply One-Hot Encoding
df_encoded = pd.get_dummies(df_standardized, columns=cols_to_encode, drop_first=False)

print("\nAFTER One-Hot Encoding:")
new_cols = [c for c in df_encoded.columns if any(orig in c for orig in cols_to_encode)]
print(f"  New columns created: {new_cols}")

print(f"\nDataset shape after encoding: {df_encoded.shape}")

In [ ]:
# Show sample of encoded data
print("\nSample of One-Hot Encoded Data:")
df_encoded[new_cols].head(10)

---
# Part III: Calculating Proximity / Correlation Analysis

Calculate similarity measures between objects and correlation between features.

## Part III(a): Calculate Similarity Measures

Calculate **Simple Matching Coefficient**, **Jaccard Similarity**, and **Cosine Similarity** between two objects.

In [ ]:
print("="*70)
print("SIMILARITY MEASURES BETWEEN TWO OBJECTS")
print("="*70)

# Select two sample objects (rows) for comparison
# Use binary/categorical columns for similarity calculation
binary_cols = [c for c in df_encoded.columns if df_encoded[c].nunique() <= 2]

# Get two sample objects
obj1 = df_encoded[binary_cols].iloc[0].values
obj2 = df_encoded[binary_cols].iloc[1].values

print(f"\nComparing Object 1 (Row 0) and Object 2 (Row 1)")
print(f"Binary features used: {len(binary_cols)} features")

In [ ]:
def simple_matching_coefficient(a, b):
    """
    Simple Matching Coefficient (SMC)
    SMC = (f11 + f00) / (f11 + f10 + f01 + f00)
    Where:
    - f11: both 1
    - f00: both 0
    - f10: a=1, b=0
    - f01: a=0, b=1
    """
    a = np.array(a).astype(int)
    b = np.array(b).astype(int)
    
    f11 = np.sum((a == 1) & (b == 1))
    f00 = np.sum((a == 0) & (b == 0))
    f10 = np.sum((a == 1) & (b == 0))
    f01 = np.sum((a == 0) & (b == 1))
    
    smc = (f11 + f00) / (f11 + f10 + f01 + f00)
    return smc, f11, f00, f10, f01

def jaccard_similarity(a, b):
    """
    Jaccard Similarity
    J = f11 / (f11 + f10 + f01)
    Ignores cases where both are 0
    """
    a = np.array(a).astype(int)
    b = np.array(b).astype(int)
    
    f11 = np.sum((a == 1) & (b == 1))
    f10 = np.sum((a == 1) & (b == 0))
    f01 = np.sum((a == 0) & (b == 1))
    
    if (f11 + f10 + f01) == 0:
        return 0
    return f11 / (f11 + f10 + f01)

def cosine_sim(a, b):
    """
    Cosine Similarity
    cos(a,b) = (a·b) / (||a|| * ||b||)
    """
    a = np.array(a).astype(float)
    b = np.array(b).astype(float)
    
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0:
        return 0
    return dot_product / (norm_a * norm_b)

print("Similarity functions defined!")

In [ ]:
# Calculate all similarity measures
print("="*60)
print("SIMILARITY CALCULATIONS")
print("="*60)

# 1. Simple Matching Coefficient
smc, f11, f00, f10, f01 = simple_matching_coefficient(obj1, obj2)
print(f"\n1. SIMPLE MATCHING COEFFICIENT (SMC):")
print(f"   f11 (both 1): {f11}")
print(f"   f00 (both 0): {f00}")
print(f"   f10 (a=1, b=0): {f10}")
print(f"   f01 (a=0, b=1): {f01}")
print(f"   SMC = (f11 + f00) / total = ({f11} + {f00}) / {f11+f00+f10+f01}")
print(f"   SMC = {smc:.4f}")

# 2. Jaccard Similarity
jaccard = jaccard_similarity(obj1, obj2)
print(f"\n2. JACCARD SIMILARITY:")
print(f"   J = f11 / (f11 + f10 + f01) = {f11} / ({f11} + {f10} + {f01})")
print(f"   Jaccard Similarity = {jaccard:.4f}")

# 3. Cosine Similarity
cos_sim = cosine_sim(obj1, obj2)
print(f"\n3. COSINE SIMILARITY:")
print(f"   cos(a,b) = (a·b) / (||a|| * ||b||)")
print(f"   Cosine Similarity = {cos_sim:.4f}")

In [ ]:
# Summary table
print("\n" + "="*60)
print("SIMILARITY MEASURES SUMMARY")
print("="*60)

similarity_df = pd.DataFrame({
    'Measure': ['Simple Matching Coefficient', 'Jaccard Similarity', 'Cosine Similarity'],
    'Value': [smc, jaccard, cos_sim],
    'Interpretation': [
        'Considers both matches (1-1 and 0-0)',
        'Only considers positive matches (1-1)',
        'Measures angle between vectors'
    ]
})
print(similarity_df.to_string(index=False))

## Part III(b): Calculate Correlation Between Features

Calculate correlation between **Commute Distance** and **Yearly Income**.

In [ ]:
print("="*60)
print("CORRELATION ANALYSIS")
print("="*60)

# Check if CommuteDistance exists, if not use available numeric columns
if 'CommuteDistance' in df_selected.columns:
    feature1 = 'CommuteDistance'
else:
    feature1 = 'NumberCarsOwned'  # Alternative feature

feature2 = 'YearlyIncome'

# Calculate Pearson correlation
correlation = df_selected[feature1].corr(df_selected[feature2])

print(f"\nCorrelation between {feature1} and {feature2}:")
print(f"Pearson Correlation Coefficient: {correlation:.4f}")

# Interpretation
if abs(correlation) < 0.3:
    strength = "Weak"
elif abs(correlation) < 0.7:
    strength = "Moderate"
else:
    strength = "Strong"

direction = "positive" if correlation > 0 else "negative"

print(f"\nInterpretation: {strength} {direction} correlation")

In [ ]:
# Correlation matrix for all numerical features
print("\nFull Correlation Matrix (Numerical Features):")
corr_matrix = df_selected[numerical_features].corr()
print(corr_matrix.round(3))

In [ ]:
# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', square=True, linewidths=0.5)
plt.title('Correlation Matrix - Numerical Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Summary

This notebook completed all parts of Assignment 2:

**Part I - Feature Selection & Preprocessing:**
- (a) Examined attributes and removed unnecessary ones
- (b) Created new DataFrame with selected features
- (c) Classified data types (Discrete/Continuous, Nominal/Ordinal/Interval/Ratio)

**Part II - Data Transformation:**
- (a) Handled null values using mean/mode imputation
- (b) Applied Min-Max normalization
- (c) Applied discretization (binning) to continuous features
- (d) Applied standardization (Z-score normalization)
- (e) Applied one-hot encoding to categorical features

**Part III - Similarity & Correlation:**
- (a) Calculated Simple Matching, Jaccard, and Cosine similarity between objects
- (b) Calculated correlation between numerical features